# Import

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import copy
import math
import random
from glob import glob

import torch
import torchvision
from torchvision import datasets, models
from torchvision.utils import draw_bounding_boxes, make_grid
from torchvision.transforms import transforms as T
from torch.utils.data import Dataset, DataLoader

from pytorch_lightning import seed_everything, LightningModule, Trainer
from sklearn.metrics import classification_report
from pytorch_lightning.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    LearningRateMonitor,
)
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from torchsummary import summary


from pycocotools.coco import COCO
from roboflow import Roboflow

import albumentations as A
from albumentations.pytorch import ToTensorV2

# Preparation
### Load Data from Roboflow

In [ ]:
if not os.path.exists("maskdetection-3"):
    print("Downloading dataset...")
    rf = Roboflow(api_key="RLpF5qnVG3u4wi0Hgkmg")
    project = rf.workspace("diza-febriyan-hasal").project("maskdetection-tdrvn")
    dataset = project.version(3).download("coco")

### Coret-coretan MCT

In [ ]:
coco = COCO(os.path.join("maskdetection-3", "train", "_annotations.coco.json"))
anotasi_img = coco.loadAnns(coco.getAnnIds(0))
print(anotasi_img)
print(anotasi_img[0]["bbox"])

### Data Reader

In [ ]:
class MaskDetection(Dataset):

    """
    MaskDetection Datareader
    Return:
        - image pixels as tensor
        - annotation as dictionary
    """

    def __init__(self, root, split="train"):
        self.root = root
        self.split = split
        self.transforms = ToTensorV2()
        self.coco = COCO(os.path.join(root, split, "_annotations.coco.json"))
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, idx: int):
        id = self.ids[idx]
        anotasi_img = self.coco.loadAnns(self.coco.getAnnIds(idx))
        imgpath = self.coco.loadImgs(id)[0]["file_name"]
        image = cv.imread(os.path.join(self.root, self.split, imgpath))
        image = cv.normalize(image, None, 0, 255, cv.NORM_MINMAX)
        # image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)

        image = self.transforms(image=image)["image"]

        bbox = [
            anotasi_img[0]["bbox"][0],
            anotasi_img[0]["bbox"][1],
            anotasi_img[0]["bbox"][2],
            anotasi_img[0]["bbox"][3],
        ]
        bbox = torch.as_tensor(bbox, dtype=torch.float32)

        return image, bbox

    def __len__(self):
        return len(self.ids)

In [ ]:
dataset_path = "maskdetection-3"
train_dataset = MaskDetection(root=dataset_path, split="train")
val_dataset = MaskDetection(root=dataset_path, split="valid")
test_dataset = MaskDetection(root=dataset_path, split="test")

## Cek Ombak DataReader dan Data Loader
> Sengaja di random, supaya gambarnya ganti-ganti

- Bounding box digambar pakai `import matplotlib.patches as patches` supaya lebih gampang
- Format bounding box mengikuti format asli dari COCO yaitu `[x, y, width, height]`

### DataReader Cek

In [ ]:
image_sample = train_dataset[random.randint(0, len(train_dataset))]
image_pixels, bbox = image_sample
print(type(image_pixels))

plt.imshow(image_pixels.permute(1, 2, 0), cmap="gray")

rect = patches.Rectangle(
    (bbox[0], bbox[1]),
    bbox[2],
    bbox[3],
    linewidth=2,
    edgecolor="r",
    facecolor="none",
)
plt.gca().add_patch(rect)
plt.show()

### Data Loader Cek


In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
    collate_fn=lambda x: tuple(zip(*x)),
    pin_memory=True,
)

print(len(train_loader))

image, bbox = next(iter(train_loader))
print(f"Image shape: {image[2].shape}")
print(f"Bbox sample: {bbox[0]}")

In [ ]:
plt.figure()
grid_img = torchvision.utils.make_grid(image, 4, 2)
plt.imshow(grid_img.permute(1, 2, 0))
plt.show()

# Pytorch Lightning

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = (
    torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, 1)
)
print(model)

In [ ]:
class OurClass(LightningModule):
    def __init__(self):
        super().__init__()

        # model architec
        self.model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
            weights="DEFAULT"
        )
        self.in_features = self.model.roi_heads.box_predictor.cls_score.in_features
        self.model.roi_heads.box_predictor = (
            self.torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
                self.in_features, 1
            )
        )

        # param

        # loss function

        # evaluation metrics

        # loss curve and accuracy curve

        # load data

        # split data

    def forward(self, x):
        return x

    def configure_optimizers(self):
        pass

    def train_dataloader(self):
        pass

    def training_step(self, batch, batch_idx):
        pass

    def training_epoch_end(self, outputs):
        pass

    def val_dataloader(self):
        pass

    def validation_step(self, batch, batch_idx):
        pass

    def validation_epoch_end(self, outputs):
        pass